# MITRE ATT&CK Group Analysis

This notebook provides an interactive analysis of MITRE ATT&CK groups. It loads the ATT&CK dataset, allows you to select a group, and displays detailed information about the selected group, including techniques used and related analysis and summary files. Additionally, it identifies the most common techniques across all groups and provides general research topics for further exploration.

## Load dependancies

In [6]:
import os
import json
from collections import Counter
from mitreattack.stix20 import MitreAttackData
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

In [7]:
# --- Load MITRE ATT&CK Data ---
display(Markdown("## Loading ATT&CK Data"))

dataset_path = "data/enterprise-attack.json"

if not os.path.exists(dataset_path):
    display(Markdown("**Error: enterprise-attack.json not found. Please ensure the file is in the 'data' directory.**"))
else:
    mitre_attack_data = MitreAttackData(dataset_path)
    groups = mitre_attack_data.get_groups(remove_revoked_deprecated=True)
    display(Markdown(f"**Retrieved {len(groups)} ATT&CK groups.**"))

## Loading ATT&CK Data

**Retrieved 159 ATT&CK groups.**

## Select a Group

In [8]:
group_names = [group.name for group in groups]
selected_group = widgets.Dropdown(
    options=group_names,
    description='Select Group:',
    layout=widgets.Layout(width='50%')
)

display(selected_group)

output = widgets.Output()
display(output)

Dropdown(description='Select Group:', layout=Layout(width='50%'), options=('Indrik Spider', 'LuminousMoth', 'W…

Output()

## Display Group Info

In [9]:
def search_related_data(group_name):
    """Searches for related analysis and summary files."""
    with output:
        clear_output(wait=False)
        display(Markdown("## Searching for Related Data"))

        analysis_folder = "MITRE_ATT&CK_Analysis"
        summary_folder = "MITRE_ATT&CK_Summaries"

        related_analysis = []
        related_summaries = []

        group = next((g for g in groups if g.name == group_name), None)
        if not group:
            display(Markdown("**Selected group not found.**"))
            return

        techniques_used_by_group = mitre_attack_data.get_techniques_used_by_group(group.id)
        mitre_techniques_used_by_group = [
            mitre_attack_data.get_attack_id(t["object"].id) for t in techniques_used_by_group if t["object"]
        ]

        # Search for analysis files
        for root, _, files in os.walk(analysis_folder):
            for file in files:
                if file.endswith(".json"):
                    file_path = os.path.join(root, file)
                    with open(file_path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                        if data.get("id") in mitre_techniques_used_by_group:
                            related_analysis.append(file_path)

        # Search for summary files
        for root, _, files in os.walk(summary_folder):
            for file in files:
                if file.endswith(".md"):
                    file_path = os.path.join(root, file)
                    for attack_id in mitre_techniques_used_by_group:
                        if attack_id in file:
                            related_summaries.append(file_path)
                            break

        display(Markdown("### Related Analysis Files"))
        if related_analysis:
            display(Markdown("\n".join([f"- `{a}`" for a in related_analysis])))
        else:
            display(Markdown("**No related analysis files found.**"))

        display(Markdown("### Related Summary Files"))
        if related_summaries:
            display(Markdown("\n".join([f"- `{s}`" for s in related_summaries])))
        else:
            display(Markdown("**No related summary files found.**"))

widgets.interactive(search_related_data, group_name=selected_group)

interactive(children=(Dropdown(description='Select Group:', layout=Layout(width='50%'), options=('Indrik Spide…

## Generate and Search MITRE Data

In [10]:
def search_related_data(group_name):
    """Searches for related analysis and summary files."""
    with output:
        clear_output(wait=False)
        display(Markdown("## Searching for Related Data"))

        analysis_folder = "MITRE_ATT&CK_Analysis"
        summary_folder = "MITRE_ATT&CK_Summaries"

        related_analysis = []
        related_summaries = []

        group = next((g for g in groups if g.name == group_name), None)
        if not group:
            display(Markdown("**Selected group not found.**"))
            return

        techniques_used_by_group = mitre_attack_data.get_techniques_used_by_group(group.id)
        mitre_techniques_used_by_group = [
            mitre_attack_data.get_attack_id(t["object"].id) for t in techniques_used_by_group if t["object"]
        ]

        # Search for analysis files
        for root, _, files in os.walk(analysis_folder):
            for file in files:
                if file.endswith(".json"):
                    file_path = os.path.join(root, file)
                    with open(file_path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                        if data.get("id") in mitre_techniques_used_by_group:
                            related_analysis.append(file_path)

        # Search for summary files
        for root, _, files in os.walk(summary_folder):
            for file in files:
                if file.endswith(".md"):
                    file_path = os.path.join(root, file)
                    for attack_id in mitre_techniques_used_by_group:
                        if attack_id in file:
                            related_summaries.append(file_path)
                            break

        display(Markdown("### Related Analysis Files"))
        if related_analysis:
            display(Markdown("\n".join([f"- `{a}`" for a in related_analysis])))
        else:
            display(Markdown("**No related analysis files found.**"))

        display(Markdown("### Related Summary Files"))
        if related_summaries:
            display(Markdown("\n".join([f"- `{s}`" for s in related_summaries])))
        else:
            display(Markdown("**No related summary files found.**"))

widgets.interactive(search_related_data, group_name=selected_group)

interactive(children=(Dropdown(description='Select Group:', layout=Layout(width='50%'), options=('Indrik Spide…

## General research topics

In [ ]:
# --- Identify Most Common Techniques ---
display(Markdown("## Most Common Techniques Across Groups"))

technique_counter = Counter()

# Iterate through all groups and collect techniques
for group in groups:
    techniques = mitre_attack_data.get_techniques_used_by_group(group.id)
    for t in techniques:
        if t["object"]:
            technique = t["object"]
            attack_id = mitre_attack_data.get_attack_id(technique.id)
            technique_counter[attack_id] += 1  # Count occurrences

# Get the top 50 most common techniques
most_common_techniques = technique_counter.most_common(50)

if most_common_techniques:
    display(Markdown("### Top 50 Most Common Techniques"))
    technique_list = ""
    
    # Retrieve technique names correctly
    all_techniques = mitre_attack_data.get_techniques(remove_revoked_deprecated=True)
    attack_id_to_name = {mitre_attack_data.get_attack_id(t.id): t.name for t in all_techniques}

    for attack_id, count in most_common_techniques:
        technique_name = attack_id_to_name.get(attack_id, "Unknown Technique")
        technique_list += f"- {technique_name} ({attack_id}) - Used by {count} groups\n"

    display(Markdown(f"```\n{technique_list}\n```"))
else:
    display(Markdown("**No techniques found across groups.**"))


## Most Common Techniques Across Groups

### Top 10 Most Common Techniques

```
- Ingress Tool Transfer (T1105) - Used by 82 groups
- Malicious File (T1204.002) - Used by 82 groups
- Spearphishing Attachment (T1566.001) - Used by 77 groups
- PowerShell (T1059.001) - Used by 76 groups
- Tool (T1588.002) - Used by 71 groups
- Windows Command Shell (T1059.003) - Used by 69 groups
- Match Legitimate Name or Location (T1036.005) - Used by 57 groups
- Web Protocols (T1071.001) - Used by 53 groups
- System Information Discovery (T1082) - Used by 53 groups
- Registry Run Keys / Startup Folder (T1547.001) - Used by 52 groups

```